# **Extração de emoções e Análise de Sentimentos em Letras de músicas do Forró**

In [2]:
import pandas as pd

In [72]:
#!pip install -r requirements.txt

In [75]:
df = pd.read_csv('./lyrics.csv', sep=';', encoding="utf-8")
df.shape

(13405, 7)

# Preprocessing

1. Remoção dos títulos das músicas que contenham os caracteres '()' 

In [77]:
df['title'].replace("\((.*?)\)", '', regex=True, inplace=True)



2. Remoção das músicas duplicadas a partir do título

In [78]:
df['title'] = df['title'].str.lower()
df['title'] = df['title'].str.strip()

titles = [i for i in df['title']]

duplicates = [i for i in titles if titles.count(i) > 1]
print(f'Quantidade de duplicatas: {len(duplicates)}')

Quantidade de duplicatas: 3469


In [79]:
df = df[~df['title'].isin(duplicates)]
print(f'Quantidade de músicas após remoção das duplicatas: {df.id.count()}')

Quantidade de músicas após remoção das duplicatas: 9885


3. Remoção das músicas sem data

In [80]:
df = df[df['year'] > 0]

print(f'Quantidade de músicas com data: {df.id.count()}')

Quantidade de músicas com data: 8139


4. Remoção dos registros sem letras

In [82]:
df = df[~df['lyric'].isnull()]
print(f'Quantidade de registros com letras: {df.lyric.count()}')

Quantidade de registros com letras: 8139


### Carregando dataframes

In [3]:
#df= pd.read_csv('./lyrics_train.csv', encoding='utf-8')
"""
Lexicon source is (C) 2016 National Research Council Canada (NRC) 
and library is for research purposes only.  
Source: http://sentiment.nrc.ca/lexicons-for-research/
"""
lexicon = pd.read_csv('./pt-nrc-lexicon.csv', sep=';', encoding='utf-8')

In [83]:
#removendo ruídos das letras
df['clean_lyric'] = df['lyric'].apply(clean_lyrics)
df['clean_lyric'].to_csv('clean_lyrics.csv', index=False)


In [84]:
df[['lyric', 'clean_lyric']]

,lyric,clean_lyric
2359,Tenho mais o que fazer Não discuto com teimoso...,discuto teimoso perder precioso viver joão tei...
2360,Morena sereia Que à beira-mar não passeia Que ...,morena sereia beiramar passeia senta praia dei...
2361,"Rio, querida Guanabara Eu sou gente também Ace...",rio querida guanabara gente aceito parabéns pa...
2362,Depois do baião foi que veio o forró Depois do...,baião veio forró forró veio rojão pisada chão ...
2363,Beira mar beira do rio Ê ê beira mar Cheguei a...,beira mar beira rio ê ê beira mar cheguei beir...
...,...,...
13397,Sabe porque não saio da tua mente? É que na ca...,saio mente cama diferente consegue largar amor...
13398,Tá faltando o quê? Pra tu ligar e eu atender A...,tá faltando pra ligar atender agiliza aê cuida...
13399,Avisa que ela voltou Tá online na parada Ficou...,avisa voltou tá online parada ficou solteirinh...
13402,Vai começar de novo Vai quebrar a cara Depois ...,começar quebrar cara tá ligando chorando casa ...


### Funções utilitárias

In [4]:
#carregando lista de acordes para remoção dentro das letras
chords_df = pd.read_csv('./chords.txt')
chords_dicty = chords_df.to_dict()
chords = set()

for i in chords_dicty['chord']:
    chords.add(chords_dicty['chord'][i].strip())
    
#função para remoção de acordes dentro de um texto
def remove_chords(text):
    txt_list = text.split()
    return (' ').join(
            word.strip()
            for word in txt_list
            if word.strip() not in chords
        )    

In [70]:
import string
import re
import spacy

nlp = spacy.load('pt_core_news_sm')

def clean_lyrics(text):
  txt = ''
  #removendo acordes
  txt = remove_chords(text)
  #removendo tags de repetição 2x, 4x, etc
  txt = txt.replace('[', '(').replace(']', ')')
  txt = re.sub(r'\([\d]x\)', '', txt)
  #removendo palavras de orientação musical
  txt = re.sub(r'\((.*?)\)', '', txt)  
  #removendo pontuacao
  txt = txt.translate(str.maketrans("", "", string.punctuation))
  #caracteres em minusculo   
  txt = txt.lower()
  #remove stop words
  doc = nlp(txt)
  txt = ' '.join(token.text 
                  for token in doc 
                  if not token.is_stop
                )
    
  return txt

In [50]:
from enelvo.normaliser import Normaliser

def normalize_text(text):
    norm = Normaliser(tokenizer='readable')
    return norm.normalise(text)